In [66]:
import pandas as pd
import re
import json
import datetime
import pandas as pd
import numpy as np
import math
import nltk
import datetime
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bindu_pamidi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bindu_pamidi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bindu_pamidi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [67]:
df = pd.read_excel(r'C:\Finacial Force\teja_joblevel\data with job level.xlsx')

df['Title'] = df['Title'].astype(str)
df['designation_abbr_str'] = df['designation_abbr_str'].astype(str)
df=df[['Contact ID 18', 'Title', 'Created Date', 'designation_abbr_str',
       'designation', 'designation_count']]

In [68]:
df.columns

Index(['Contact ID 18', 'Title', 'Created Date', 'designation_abbr_str',
       'designation', 'designation_count'],
      dtype='object')

In [69]:
with open (r'C:\Finacial Force\job-titles.json',"r") as f:
    data = json.loads(f.read())
df_jb = pd.DataFrame(data)
df_jb['designation'] = df_jb['job-titles'].apply(lambda x: re.split(" |&|,",x)[-1])
jb = set(df_jb['designation'].unique())
df_jb['designation'].nunique()

5402

In [70]:
df['designation'] = df['designation_abbr_str'].apply(lambda x: list(set(x.split(" "))&jb))
df['designation_count'] = df['designation'].apply(lambda x:len(x))

In [71]:
df['designation_count'].sum()

973364

In [72]:
jb_list = [item for sublist in df['designation'].to_list() for item in sublist]
len(jb_list)

973364

In [73]:
df_designation = pd.DataFrame({'jb_list':jb_list})
df_designation = df_designation.reset_index().groupby('jb_list').count().reset_index().rename(columns = {'index':'count'}).sort_values(by = 'count', ascending = False).reset_index(drop = True)

In [74]:
df_designation[df_designation['count']<1000].to_clipboard()

In [75]:
df_designation_top = pd.read_clipboard()
# df_designation_top=df_designation[df_designation['count']<1000]
df_designation_top,df_designation_top['count'].sum()

(           jb_list  count
 127      operation    996
 128         public    983
 129          audit    977
 130       programs    968
 131         member    962
 ...            ...    ...
 2300           huc      1
 2301        hunter      1
 2302  hydrographic      1
 2303     sociology      1
 2304           zoo      1
 
 [2178 rows x 2 columns],
 131096)

In [76]:
# jb_top = list(df_designation_top['jb_list'])
jb_top=['director',
 'president',
 'manager',
 'chief',
 'officer',
 'head',
 'executive',
 'consultant',
 'lead',
 'partner',
 'assistant',
 'analyst',
 'associate',
 'managing',
 'professional',
 'engineer',
 'administrator',
 'controller',
 'principal',
 'leader',
 'architect',
 'founder',
 'coordinator',
 'developer',
 'accountant',
 'advisor',
 'supervisor',
 'deputy',
 'staff',
 'chairman',
 'technician',
 'representative',
 'treasurer',
 'secretary',
 'employee',
 'strategist',
 'professor',
 'instructor',
 'designer',
 'coach',
 'chairperson',
 'recruiter',
 'investor',
 'partners',
 'contractor',
 'provider',
 'planner',
 'scientist',
 'student',
 'supplier',
 'trainer',
 'auditor',
 'producer',
 'directors',
 'doctor',
 'technologist',
 'merchant',
 'clerk',
 'physician',
 'chancellor',
 'attorney',
 'personnel',
 'researcher',
 'volunteer',
 'payer',
 'worker',
 'managers',
 'entrepreneur']

In [77]:
df['designation_top'] = df['designation'].apply(lambda x: [i for i in jb_top if i in x])
df['designation_top_one'] = df['designation_top'].apply(lambda x: x[0] if len(x)>0 else None)
df['designation_top_count'] = df['designation_top'].apply(lambda x:len(x))

In [78]:
df['designation_top_count'].value_counts()

1    158342
2     44978
0     19597
3     13139
4      4113
5       980
6       100
7         4
8         2
Name: designation_top_count, dtype: int64

In [112]:
def assign_designation(x):
    if 'chief' in x['designation_top']:
        if 'officer' in x['designation']:
            return 'C-Suite'
    if 'founder' in x['designation_top']:
        return 'C-Suite'
    if 'chairman' in x['designation_top'] or 'chairperson' in x['designation_top']:
        return 'C-Suite'
    if 'managing' in x['designation_top']:
        if 'director' in x['designation_top'] or 'partner' in x['designation_top']:
            return 'C-Suite'

    if 'president' in x['designation_top']:
        if 'vice' in x['designation']:
            if 'senior' in x['designation'] or 'sr' in x['designation']:
                return 'Senior Vice President'
            else:
                return 'Vice President'
        else:
            return 'C-Suite'
    if 'svp' in x['designation']:
        return 'Senior Vice President'

    if 'controller' in x['designation_top']:
        if 'senior' in x['designation'] or 'sr' in x['designation']:
            return 'Senior Controller'
        else:
            return 'Controller'
        
    if 'director' in x['designation_top'] or 'directors' in x['designation_top']:
        if 'senior' in x['designation'] or 'sr' in x['designation']:
            return 'Senior Director'
        else:
            return 'Director'
    if 'head' in x['designation_top']:
        if 'senior' in x['designation'] or 'sr' in x['designation']:
            return 'Senior Director'
        else:
            return 'Director'
        
    if 'leader' in x['designation_top']:
        #add rule based on the functional area : If the area is support domain, this can Other Independent Contributors
        if 'senior' in x['designation'] or 'sr' in x['designation']:
            return 'Senior Director'
        else:
            return 'Director'

    if 'manager' in x['designation_top'] or 'managers' in x['designation_top']:
        if 'senior' in x['designation'] or 'sr' in x['designation']:
            return 'Senior Manager'
        else:
            return 'Manager'
        
    if 'lead' in x['designation_top']:
        if 'team' in x['designation']:
            return 'Manager'
        
    #Can change the support functional areas after function identification.
    if len(set([
         'consultant',#provide expert opinions, analysis, recommendations to organizations or individuals
         'analyst',
         'lead',
         'executive',
         'partner',
         'partners',
         'chief',
         'architect',
         'associate',
         'principal',
         'accountant',
         'engineer',
         'advisor',
         'strategist',
         'entrepreneur'
         ])&set(x['designation']))>0:
            return 'Analyst'
    if len(set([
        'administrator',
        'coordinator',
        'officer',
        'supervisor',
        'assistant',
        'developer',
        'professional',
        'representative',
        'technician',
        'employee', 'clerk', 'attorney', 'doctor', 'staff', 'contractor',
       'treasurer', 'investor', 'designer', 'managing', 'planner',
       'scientist', 'trainer', 'producer', 'chancellor', 'instructor',
       'researcher', 'recruiter', 'professor', 'worker', 'coach',
       'secretary', 'technologist', 'auditor', 'deputy', 'student',
       'physician', 'personnel', 'volunteer', 'provider', 'supplier',
       'merchant', 'payer','operator',
        'support','specialist','intern'
        ])&set(x['designation']))>0:
            return 'Other Independent Contributor'
#     if len(set([
#          'business',
#         'sales',
#         'finance',
#         'marketing',
#         'product',
#         'strategy',
#         'solutions',
#         'accounting',
#         'consulting',
#         'enterprise',
#         'financial',
#         'strategic',
#         'accounts'
#          ])&set(x['designation']))>0:
#             return 'Analyst'
#     if len(set([
#          'operations',
#         'technology',
#         'management',
#         'information',
#         'development',
#         'service',
#         'customer',
#         'systems',
#         'engineering',
#         'resources',
#         'cloud',
#         'intern',
#         'technical',
#         'administration',
#         'billing',
#         'chef',
#         'procurement',
#         'research',
#         'security',
#         'services'
#          ])&set(x['designation']))>0:
#             return 'Other Independent Contributor'  
    return None

In [113]:
df['job_level'] = df.apply(lambda x: assign_designation(x), axis = 1)

In [114]:
df['job_level'].value_counts()

Director                         63913
Manager                          33524
Vice President                   32339
C-Suite                          26661
Analyst                          26482
Other Independent Contributor    13695
Senior Director                   9566
Senior Vice President             8908
Senior Manager                    8642
Controller                        2153
Senior Controller                  224
Name: job_level, dtype: int64

In [115]:
df['job_level'].value_counts()

Director                         63913
Manager                          33524
Vice President                   32339
C-Suite                          26661
Analyst                          26482
Other Independent Contributor    13695
Senior Director                   9566
Senior Vice President             8908
Senior Manager                    8642
Controller                        2153
Senior Controller                  224
Name: job_level, dtype: int64

In [116]:
df.isnull().sum()

Contact ID 18                0
Title                        0
Created Date                 0
designation_abbr_str         0
designation                  0
designation_count            0
designation_top              0
designation_top_one      19597
designation_top_count        0
job_level                15148
functional_list              0
functional_list1             0
dtype: int64

In [29]:
8125/len(df)

0.03367805848583449

In [33]:
df[(df['job_level'].isnull())&df['designation'].isin([['enterprise']])]

,Contact ID 18,Title,Created Date,designation_abbr_str,designation,designation_count,designation_top,designation_top_one,designation_top_count,job_level


In [83]:
jb_list_left = [item for sublist in df[(df['job_level'].isnull())]['designation'].to_list() for item in sublist]
len(jb_list_left)

30963

In [84]:
df_designation_left = pd.DataFrame({'jb_list':jb_list_left})
df_designation_left = df_designation_left.reset_index().groupby('jb_list').count().reset_index().rename(columns = {'index':'count'}).sort_values(by = 'count', ascending = False).reset_index(drop = True)

In [36]:
df_designation_left.to_clipboard()

In [37]:
df[(df['job_level'].isnull())&~(df['designation_top_one'].isnull())&(df['designation_top_one']=='managing')]

,Contact ID 18,Title,Created Date,designation_abbr_str,designation,designation_count,designation_top,designation_top_one,designation_top_count,job_level


In [39]:
df[df['designation_top_one'].isnull()]

,Contact ID 18,Title,Created Date,designation_abbr_str,designation,designation_count,designation_top,designation_top_one,designation_top_count,job_level
2,0033000000FArFRAA1,nan,2006-01-17,nan,[],0,[],None,0,None
5,0033000000RRjX5AAL,Operator,2008-01-21,operator,[operator],1,[],None,0,Other Independent Contributor
10,0033000000TOfg0AAD,General Manageme,2008-05-29,general manageme,[general],1,[],None,0,None
12,0033000000TPBrsAAH,Mr,2008-06-02,mr,[],0,[],None,0,None
37,0033000000UJWVMAA5,SE,2008-09-02,se,[],0,[],None,0,None
...,...,...,...,...,...,...,...,...,...,...
241203,0034Q00003GDfAUQA1,Team Member Operational Support,2022-04-07,support team member operational,"[member, support, team, operational]",4,[],None,0,Other Independent Contributor
241241,0034Q00003GDn7uQAD,Directeur IT opérationnel,2022-04-08,opérationnel information technology directeur,"[technology, information]",2,[],None,0,Other Independent Contributor
241242,0034Q00003GDkfMQAT,Commercial Finance - Technology & Innovation,2022-04-08,finance commercial technology innovation,"[finance, commercial, technology]",3,[],None,0,Analyst
241247,0034Q00003GDl3gQAD,Signee,2022-04-08,signee,[],0,[],None,0,None


In [40]:
df[df['designation_count']==0]

,Contact ID 18,Title,Created Date,designation_abbr_str,designation,designation_count,designation_top,designation_top_one,designation_top_count,job_level
2,0033000000FArFRAA1,nan,2006-01-17,nan,[],0,[],None,0,None
12,0033000000TPBrsAAH,Mr,2008-06-02,mr,[],0,[],None,0,None
37,0033000000UJWVMAA5,SE,2008-09-02,se,[],0,[],None,0,None
39,0033000000UKrFOAA1,Left,2008-09-12,left,[],0,[],None,0,None
45,0033000000UMH74AAH,"RVP, ESB APAC",2008-10-02,esb apac rvp,[],0,[],None,0,None
...,...,...,...,...,...,...,...,...,...,...
240777,0034Q00003FWV7cQAH,Advokatfirmaet Simonsen Vogt Wiig AS,2022-04-01,wiig advokatfirmaet as vogt simonsen,[],0,[],None,0,None
241090,0034Q00003GDCxxQAH,"Se, Sfs - Cmt",2022-04-05,sfs cmt se,[],0,[],None,0,None
241095,0034Q00003GD28bQAD,Growth PO,2022-04-05,growth po,[],0,[],None,0,None
241134,0034Q00003GD6I3QAL,Renewals,2022-04-05,renewals,[],0,[],None,0,None


In [ ]:
df.to_excel('data with job level.xlsx',index=False)

# Logic for Functional words

In [126]:
df_functional_words=pd.read_excel(r"C:\Finacial Force\teja_joblevel\job_level_count.xlsx",sheet_name='functionalwords',header=1)
list(df_functional_words['functionalwords'])

['desktop',
 'accounts',
 'developer',
 'operations',
 'controller',
 'technology',
 'service',
 'training',
 'engineer',
 'web',
 'market',
 'project',
 'information technology',
 'bookkeeper',
 'engineering',
 'consultant',
 'accountant',
 'accounting',
 'finance',
 'field service',
 'internet',
 'compensation',
 'sales',
 'mechatronic',
 'resource',
 'information',
 'sourcing',
 'project management',
 'recruiting',
 'procurement',
 'marketing',
 'ops',
 'management',
 'human',
 'full stack',
 'computer',
 'Stockholders',
 'supplier',
 'suppliers',
 'bank',
 'banks',
 'transaction',
 'transactions',
 'reporting',
 'financial',
 'vendors',
 'invoices',
 'Employee Relations',
 'Employee Engagement',
 'L&D',
 'Generalist',
 'HRBP',
 'Human Resources Business Partner',
 'Executive',
 'Campus Recruiting',
 'customer care',
 'customer satisfaction',
 'customer services']

In [167]:
def apply_map_job(x,word):
    word_list=set([i.lower() for i in word.split()])
    title_words=set([i.lower() for i in x.split()])
    if len(word_list-title_words)==0:
        return word+' & '
#     if word.lower() in [i.lower() for i in x]:
#         return word+' & '
    return ''
functional_words=list(df_functional_words['functionalwords'])
# functional_words=list(set(functional_words))
functional_words=list(set(functional_words)-set(jb_top))
df['functional_list']=''
for func in tqdm(functional_words):
    df['functional_list1']=''
    df['functional_list1']=df['designation_abbr_str'].apply(apply_map_job,args=(func,))
    df['functional_list']+=df['functional_list1']

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [00:28<00:00,  1.84it/s]


In [183]:
functional_words

['vendors',
 'accounts',
 'engineering',
 'mechatronic',
 'recruiting',
 'Human Resources Business Partner',
 'information',
 'Campus Recruiting',
 'customer care',
 'computer',
 'Employee Relations',
 'project',
 'management',
 'sourcing',
 'Stockholders',
 'bookkeeper',
 'service',
 'web',
 'sales',
 'financial',
 'customer services',
 'operations',
 'compensation',
 'reporting',
 'banks',
 'HRBP',
 'transactions',
 'invoices',
 'Employee Engagement',
 'desktop',
 'finance',
 'L&D',
 'accounting',
 'customer satisfaction',
 'information technology',
 'Executive',
 'full stack',
 'Generalist',
 'training',
 'technology',
 'resource',
 'transaction',
 'procurement',
 'market',
 'ops',
 'project management',
 'suppliers',
 'field service',
 'human',
 'internet',
 'marketing',
 'bank']

### now list of functional words

In [184]:
import numpy as np
l=" ".join(list(map(lambda x:' '.join(x),np.array(df['functional_list'].apply(lambda x:x.split(' & '))))))
freq_words_functional=nltk.FreqDist(l.split())
freq_words_functional

FreqDist({'technology': 34765, 'information': 30772, 'operations': 23671, 'management': 15394, 'project': 14521, 'Executive': 13637, 'finance': 12584, 'financial': 11367, 'sales': 10883, 'service': 7812, ...})

In [185]:
dict(freq_words_functional)

{'information': 30772,
 'management': 15394,
 'financial': 11367,
 'marketing': 5841,
 'operations': 23671,
 'Executive': 13637,
 'sales': 10883,
 'engineering': 3967,
 'accounts': 1617,
 'customer': 911,
 'services': 637,
 'finance': 12584,
 'technology': 34765,
 'service': 7812,
 'market': 1239,
 'procurement': 1410,
 'accounting': 3827,
 'project': 14521,
 'sourcing': 588,
 'human': 5861,
 'care': 248,
 'web': 427,
 'resource': 2790,
 'compensation': 415,
 'training': 1044,
 'bookkeeper': 46,
 'reporting': 1337,
 'computer': 192,
 'desktop': 103,
 'recruiting': 463,
 'internet': 115,
 'transaction': 147,
 'field': 220,
 'invoices': 15,
 'bank': 187,
 'Generalist': 110,
 'transactions': 64,
 'full': 27,
 'stack': 27,
 'satisfaction': 26,
 'Human': 150,
 'Resources': 150,
 'Business': 150,
 'Partner': 150,
 'Employee': 126,
 'Relations': 89,
 'Engagement': 37,
 'vendors': 5,
 'suppliers': 8,
 'Campus': 8,
 'Recruiting': 8,
 'banks': 19,
 'mechatronic': 1}

<b>now get frequecy of words in nonfunctional records</b>

In [170]:
# df['functional_list']
df.columns
df_nonfunctional=df[df['functional_list']=='']
df_nonfunctional.head()

,Contact ID 18,Title,Created Date,designation_abbr_str,designation,designation_count,designation_top,designation_top_one,designation_top_count,job_level,functional_list,functional_list1
0,0033000000Ev6cRAAR,Pres,2006-01-12,president,[president],1,[president],president,1,C-Suite,,
1,0033000000Ev6cSAAR,Manager,2006-01-12,manager,[manager],1,[manager],manager,1,Manager,,
2,0033000000FArFRAA1,nan,2006-01-17,nan,[],0,[],None,0,None,,
3,0033000000FWaRLAA1,Director,2006-01-27,director,[director],1,[director],director,1,Director,,
5,0033000000RRjX5AAL,Operator,2008-01-21,operator,[operator],1,[],None,0,Other Independent Contributor,,


In [171]:
#list of designations
# l_non_functional
l_non_functional=" ".join(list(map(lambda x:' '.join(x),np.array(df_nonfunctional['designation_abbr_str'].apply(lambda x:x.split())))))
# l_non_functional
freq_words_nonfunctional=nltk.FreqDist(l_non_functional.split())
freq_words_nonfunctional

FreqDist({'director': 28712, 'president': 21217, 'manager': 20243, 'vice': 19129, 'senior': 17480, 'services': 17337, 'of': 10173, 'and': 9092, 'business': 7913, 'head': 7635, ...})

<b>
    <li>we got frequecy in functional and non functional words</li>
</b>

In [172]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html
df_functiona_words_freq=pd.DataFrame(list(dict(freq_words_functional).items()) ,columns=['word','frequency'])
df_nonfunctiona_words_freq=pd.DataFrame(list(dict(freq_words_nonfunctional).items()) ,columns=['word','frequency'])
                        

In [134]:
with pd.ExcelWriter('C:/Finacial Force/functional/word_frequencyv2.0.xlsx') as writer:
    df_functiona_words_freq.to_excel(writer,sheet_name='functional words')
    df_nonfunctiona_words_freq.to_excel(writer,sheet_name='word frequecy nonfunc')

In [135]:
df[df['Contact ID 18']=='0034Q00003D9J6XQAV']

,Contact ID 18,Title,Created Date,designation_abbr_str,designation,designation_count,designation_top,designation_top_one,designation_top_count,job_level,functional_list,functional_list1
227719,0034Q00003D9J6XQAV,"( 2017 Vice President, Accounting & Finance & ...",2021-12-09,accounting 2017 finance 2012 assistant present...,"[finance, president, vice, assistant, accounting]",5,"[president, assistant]",president,2,Vice President,,


In [173]:
print("no of records which have functional key words are :",df[df['functional_list']!=''].shape)
print("no of records which dont have functional key words are :",df[df['functional_list']==''].shape)
# df[df['functional_list']!=''].shape

no of records which have functional key words are : (126483, 12)
no of records which dont have functional key words are : (114772, 12)


In [65]:
# df_nonfunctiona_words_freq[df_nonfunctiona_words_freq['frequency']>1000]
# jb_top
df_nonfunctiona_words_freq[~df_nonfunctiona_words_freq['word'].str.contains('|'.join(jb_top),regex=True)].to_excel(r'C:\Finacial Force\154functional roles\nonfunctional words no designation.xlsx',index=False)

## non functional words dataframe

In [174]:
df_non_functional=df[df['functional_list']=='']
print(df_non_functional.shape)

(114772, 12)


In [175]:
df_func_100_nonfunc=pd.read_excel(r'C:\Finacial Force\154functional roles\nonfunctional words no designation.xlsx',header=1,usecols='B:D')
# with open(r'C:\Finacial Force\154functional roles\nonfunctional words no designation.xlsx') as excel:
#     df_func_100_nonfunc=pd.read_excel(excel,header=1,usecols='B:D')
print(df_func_100_nonfunc.shape)
print(df_func_100_nonfunc.columns)

(17654, 3)
Index(['word', 'frequency', 'flag'], dtype='object')


In [176]:
df_func_100_nonfunc=df_func_100_nonfunc[(df_func_100_nonfunc['frequency']>100) & (df_func_100_nonfunc['flag']==1)]
df_func_100_nonfunc.head()

,word,frequency,flag
1,services,16877,1.0
5,business,7308,1.0
7,development,4976,1.0
8,solutions,3908,1.0
9,delivery,3558,1.0


In [177]:
import warnings
warnings.filterwarnings("ignore")

df_non_functional.head()
def apply_map_nonfunc_word(x,word):
    word_list=set([i.lower() for i in word.split()])
    title_words=set([i.lower() for i in x.split()])
    if len(word_list-title_words)==0:
        return word+' & '
#     if word.lower() in [i.lower() for i in x]:
#         return word+' & '
    return ''
nonfunctional_words_freq=list(df_func_100_nonfunc['word'])
nonfunctional_words_freq=list(set(nonfunctional_words_freq))
df_non_functional['non_functional_list']=''
for func in tqdm(nonfunctional_words_freq):
    df_non_functional['nonfunctional_list1']=''
    df_non_functional['nonfunctional_list1']=df_non_functional['designation_abbr_str'].apply(apply_map_nonfunc_word,args=(func,))
    df_non_functional['non_functional_list']+=df_non_functional['nonfunctional_list1']

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [00:21<00:00,  4.17it/s]


In [178]:
df_non_functional[df_non_functional['non_functional_list']==''].shape
# df_non_functional.columns

(42896, 14)

In [186]:
import numpy as np
l=" ".join(list(map(lambda x:' '.join(x),np.array(df_non_functional['non_functional_list'].apply(lambda x:x.split(' & '))))))
freq_words_non100functional=nltk.FreqDist(l.split())
freq_words_non100functional

FreqDist({'services': 17330, 'business': 7911, 'development': 5158, 'solutions': 4182, 'delivery': 3629, 'customer': 3443, 'strategy': 3440, 'consulting': 3203, 'support': 2988, 'systems': 2948, ...})

In [187]:
dict(freq_words_non100functional)

{'infrastructure': 1034,
 'audit': 621,
 'technical': 2520,
 'salesforce': 2508,
 'account': 1449,
 'solution': 1410,
 'business': 7911,
 'services': 17330,
 'application': 1242,
 'specialist': 2246,
 'recruitment': 224,
 'development': 5158,
 'administrative,': 468,
 'administration,': 468,
 'billing': 413,
 'team': 1521,
 'delivery': 3629,
 'operation': 624,
 'strategy': 3440,
 'editor': 111,
 'solutions': 4182,
 'design': 735,
 'oracle': 206,
 'operating': 1577,
 'logistics': 364,
 'program': 1923,
 'production': 527,
 'customer': 3443,
 'support': 2988,
 'strategic': 1909,
 'systems': 2948,
 'insurance': 316,
 'software': 1387,
 'sap': 441,
 'cloud': 1460,
 'consulting': 3203,
 'relations': 839,
 'content': 389,
 'manufacturing': 327,
 'digital': 2089,
 'investments': 146,
 'programmer': 140,
 'legal': 436,
 'revenue': 845,
 'tax': 776,
 'medical': 394,
 'analytics': 1462,
 'clinical': 661,
 'planning': 1562,
 'operational': 481,
 'healthcare': 466,
 'consumer': 303,
 'insights': 2

In [180]:
df_non_functional.shape

(114772, 14)

In [179]:
with pd.ExcelWriter(r'C:\Finacial Force\184functional\job title functional.xlsx') as writer:
    df.to_excel(writer,sheet_name='functional words')
    df_non_functional.to_excel(writer,sheet_name='non functional 2')

# Load tejaswinis data

In [ ]:
df=pd.read_excel("C:/Finacial Force/teja_joblevel/data with job level.xlsx")

In [ ]:
print(df.columns)

In [ ]:
df.groupby(['job_level']).count()

In [ ]:
# df['job_level'].unique()
# np.isnan(df['job_level'].unique())
# list(df['job_level'].unique()[2])
l= list(df['job_level'].unique())
l.remove(np.nan)
l

In [ ]:
#generate random 100 records
#https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html
import numpy as np
with pd.ExcelWriter('C:/Finacial Force/teja_joblevel/random data each with job level.xlsx') as writer:
    job_levels=list(df['job_level'].unique())
    job_levels.remove(np.nan)
    for job_level in job_levels:
        rand_list=np.random.randint(low=0,high=df[df['job_level']==job_level].shape[0]-1,size=100)
        df[df['job_level']==job_level].iloc[rand_list].to_excel(writer,sheet_name=job_level)
#     rand_list=np.random.randint(low=0,high=df.shape[0]-1,size=100)
#     df.iloc[rand_list].to_excel(writer,sheet_name='random 1')
#     rand_list=np.random.randint(low=0,high=df.shape[0]-1,size=100)
#     df.loc[rand_list].to_excel(writer,sheet_name='random 2')

# Filterout unwanted job titles based on conditions

In [ ]:
df['Title']=df['Title'].astype(str)
print("shape of dataframe before applying rule 1 is :",df.shape)
#rule 1
#https://www.guru99.com/python-regular-expressions-complete-tutorial.html
df['rule1']=df['Title'].apply(lambda x:True if re.search(r'directeur',x) else False)
# df=df[df['rule1']==False]
#rule 2
df['rule2']=df['Title'].apply(lambda x:x.isascii())
#rule 3
df['rule3']=df['Title'].apply(lambda x:x.isascii())

In [ ]:
df['job_level'].isna()
# df.groupby(['job_level']).count()
df[df['job_level'].isna()]

In [ ]:
df.isnull().sum()